### Workbook 3 - Extracting UrbanSound8K audio features for a Convolutional Neural Network 

First, import the libraries. As with the earlier notebooks, the audio processing is handled by a library called librosa, if you haven't already installed it on your local system, do that with: pip install librosa

You'll also need Keras and Tensorflow installed. This was updated on 23 July 2017 to use **Python3**, **Keras 2** and **Tensorflow 1.2**.

In [1]:
import glob
import os
import librosa
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np

Notebook 1 explained how to extract and aggregate several audio features using processing methods provided by the librosa library which could be fed into a Feed-Forward Network.  

This notebook is going to describe how to learn audio data using a different architecture, a Convolutional Neural Network (CNN). 

A CNN organises hidden units to take advantage of the local structure present in two-dimensional input data (the classic example being edges in images). By concentrating on identifying local features each hidden unit only needs to process a tiny part of the whole input space, instead of being connected to all the inputs coming from the previous
layer. Processing proceeds by successively considering small windows of the data set, (e.g. 3x3 pixels), called the receptive field.

The weights of hidden units create a convolutional kernel (or filter) which is applied to the whole input space, like a succession of tiles, resulting in a feature map. As a result, one set of weights can be reused for the whole input space. As a feature like an edge can occur anywhere in the input space, the CNN approach greatly reduces the number of parameters required, as well as improving the model's robustness.  

A typical convolutional layer will consist of numerous filters (feature maps). Further dimensionality reduction can be achieved through pooling layers, which merge adjacent cells of a feature map, using pooling operations such as 
max (winner takes all) or the mean of the input cells. This downsampling further improves the tolerance of the network to variation and noise.

CNNs have proved especially successful at classification tasks, particularly of images, but to use them to classify audio files, we'll need some way of extracting audio features in such a way that they reflect the kind of input data a CNN expects. The method is explained by Karol J. Piczak in his paper Environmental sound classification with convolutional neural networks (http://karol.piczak.com/papers/Piczak2015-ESC-ConvNet.pdf). This describes how to get equal size segments from varying length audio clips, and which audio features can be fed into the network as separate channels (akin to RGB channels of colour images). 


### Feature Extraction for CNNs (optional)

The idea is to calculate log-scaled mel-spectrograms and their corresponding deltas from a sound clip. Because we need fixed size input, we split each sound clip into 41 overlapping frames, one for each of the 60 mel-bands, giving an array of 60 rows and 41 columns. The mel-spectrogram for each band/segment and its time-series deltas will become two channels, which becomes our input to feed into the CNN. Other features could be calculated in the same way and supplied as a separate input channel, but we'll stick with just the mel-spectragram data for this example. 

The feature extraction process is based on example code posted by Aaqib Saeed:
http://aqibsaeed.github.io/2016-09-24-urban-sound-classification-part-2/

In [6]:
def windows(data, window_size):
    start = 0
    while start < len(data):
        yield start, start + window_size
        start += (window_size / 2)

def extract_features(parent_dir,sub_dirs,file_ext="*.wav",bands = 60, frames = 41):
    window_size = 512 * (frames - 1)
    log_specgrams = []
    labels = []
    for l, sub_dir in enumerate(sub_dirs):
        for fn in glob.glob(os.path.join(parent_dir, sub_dir, file_ext)):
            sound_clip,s = librosa.load(fn)
            label = fn.split('fold')[1].split('-')[1]        
            for (start,end) in windows(sound_clip,window_size):
                if(len(sound_clip[start:end]) == int(window_size)):
                    signal = sound_clip[start:end]
                    melspec = librosa.feature.melspectrogram(signal, n_mels = bands)
                    logspec = librosa.logamplitude(melspec)
                    logspec = logspec.T.flatten()[:, np.newaxis].T
                    log_specgrams.append(logspec)
                    labels.append(label)
            
    log_specgrams = np.asarray(log_specgrams).reshape(len(log_specgrams),bands,frames,1)
    features = np.concatenate((log_specgrams, np.zeros(np.shape(log_specgrams))), axis = 3)
    for i in range(len(features)):
        features[i, :, :, 1] = librosa.feature.delta(features[i, :, :, 0])
    
    return np.array(features), np.array(labels,dtype = np.int)

def one_hot_encode(labels):
    n_labels = len(labels)
    n_unique_labels = len(np.unique(labels))
    one_hot_encode = np.zeros((n_labels,n_unique_labels))
    one_hot_encode[np.arange(n_labels), labels] = 1
    return one_hot_encode

If the sound clip is longer than the expected window size, the extraction process will create several features rather than just one. As the sound recordings are often repetitive, this is an example of Data Augmentation, creating several training examples from a single source recording. In the example below, we can see how a sample audio file results in the creation of 7 feature rows, each consisting of 3-dimensional array (60x41x2) of data values.

This code also shows how the feature representation used for a CNN is not as compact as the inputs fed into the Feed-Forward Network, which had only 193 features. In this case, 88200 data points have been reduced to 34440. The hope is by retaining more of the source data, we are able to learn from it more subtly.

In [11]:
def extract_feature_array(filename, bands = 60, frames = 41):
    window_size = 512 * (frames - 1)
    log_specgrams = []
    sound_clip,s = librosa.load(filename)        
    for (start,end) in windows(sound_clip,window_size):
        start = int(start)
        end = int(end)
        if(len(sound_clip[start:end]) == window_size):
            signal = sound_clip[start:end]
            melspec = librosa.feature.melspectrogram(signal, n_mels = bands)
            logspec = librosa.logamplitude(melspec)
            logspec = logspec.T.flatten()[:, np.newaxis].T
            log_specgrams.append(logspec)
            
    log_specgrams = np.asarray(log_specgrams).reshape(len(log_specgrams),bands,frames,1)
    features = np.concatenate((log_specgrams, np.zeros(np.shape(log_specgrams))), axis = 3)
    for i in range(len(features)):
        features[i, :, :, 1] = librosa.feature.delta(features[i, :, :, 0])
    
    return np.array(features)

sample_filename = "samples/us8k/music.wav"
features = extract_feature_array(sample_filename)
data_points, _ = librosa.load(sample_filename)
print ("IN: Initial Data Points =", len(data_points))
print ("OUT: Total features =", np.shape(features))


IN: Initial Data Points = 88200
OUT: Total features = (7, 60, 41, 2)


### Saving Extracted Features (optional)

The code in the cell below can be run (once) to convert the raw audio files into features, which are stored as numpy arrays. As this process is quite time consuming, we'd prefer to just do it once, and then load the numpy data when we want to do some training. 

In [22]:
# use this to process the audio files into numpy arrays
def save_folds(data_dir):
    for k in range(1,11):
        fold_name = 'fold' + str(k)
        print ("\nSaving " + fold_name)
        features, labels = extract_features(parent_dir, [fold_name])
        labels = one_hot_encode(labels)
        
        print ("Features of", fold_name , " = ", features.shape)
        print ("Labels of", fold_name , " = ", labels.shape)
        
        feature_file = os.path.join(data_dir, fold_name + '_x.npy')
        labels_file = os.path.join(data_dir, fold_name + '_y.npy')
        np.save(feature_file, features)
        print ("Saved " + feature_file)
        np.save(labels_file, labels)
        print ("Saved " + labels_file)

def assure_path_exists(path):
    mydir = os.path.join(os.getcwd(), path)
    if not os.path.exists(mydir):
        os.makedirs(mydir)
        
# uncomment this to recreate and save the feature vectors
parent_dir = "raw/us8k" # Where you have saved the UrbanSound8K data set"       
save_dir = "data/us8k-np-cnn"
assure_path_exists(save_dir)
save_folds(save_dir)



Saving fold1
Features of fold1  =  (2400, 60, 41, 2)
Labels of fold1  =  (2400, 10)
Saved data/us8k-np-cnn-mini/fold1_x.npy
Saved data/us8k-np-cnn-mini/fold1_y.npy

Saving fold2
Features of fold2  =  (2076, 60, 41, 2)
Labels of fold2  =  (2076, 10)
Saved data/us8k-np-cnn-mini/fold2_x.npy
Saved data/us8k-np-cnn-mini/fold2_y.npy

Saving fold3
Features of fold3  =  (2371, 60, 41, 2)
Labels of fold3  =  (2371, 10)
Saved data/us8k-np-cnn-mini/fold3_x.npy
Saved data/us8k-np-cnn-mini/fold3_y.npy


### Loading Features

If you have a powerful computer, and don't mind the training process taking longer, you can set the all_folds flag to true - which will create a single large training set of 43722 examples, using the first 8 folds. 

In [12]:
# this will aggregate all the training data 
def load_all_folds(): 
    subsequent_fold = False
    for k in range(1,9):
        fold_name = 'fold' + str(k)
        print ("\nAdding " + fold_name)
        feature_file = os.path.join(data_dir, fold_name + '_x.npy')
        labels_file = os.path.join(data_dir, fold_name + '_y.npy')
        loaded_features = np.load(feature_file)
        loaded_labels = np.load(labels_file)
        print ("New Features: ", loaded_features.shape)

        if subsequent_fold:
            train_x = np.concatenate((features, loaded_features))
            train_y = np.concatenate((labels, loaded_labels))
        else:
            train_x = loaded_features
            train_y = loaded_labels
            subsequent_fold = True
        
    # use the penultimate fold for validation
    valid_fold_name = 'fold9'
    feature_file = os.path.join(data_dir, valid_fold_name + '_x.npy')
    labels_file = os.path.join(data_dir, valid_fold_name + '_y.npy')
    valid_x = np.load(feature_file)
    valid_y = np.load(labels_file) 

    # and use the last fold for testing
    test_fold_name = 'fold10'
    feature_file = os.path.join(data_dir, test_fold_name + '_x.npy')
    labels_file = os.path.join(data_dir, test_fold_name + '_y.npy')
    test_x = np.load(feature_file)
    test_y = np.load(labels_file)
    

# this is used to load the folds incrementally
def load_folds(folds):
    subsequent_fold = False
    for k in range(len(folds)):
        fold_name = 'fold' + str(folds[k])
        feature_file = os.path.join(data_dir, fold_name + '_x.npy')
        labels_file = os.path.join(data_dir, fold_name + '_y.npy')
        loaded_features = np.load(feature_file)
        loaded_labels = np.load(labels_file)
        print (fold_name, "features: ", loaded_features.shape)

        if subsequent_fold:
            features = np.concatenate((features, loaded_features))
            labels = np.concatenate((labels, loaded_labels))
        else:
            features = loaded_features
            labels = loaded_labels
            subsequent_fold = True
        
    return features, labels

-----

### Training a Convolutional Neural Network with Keras and TensorFlow

First, the imports we need and a few configuration variables.

In [13]:
tf.set_random_seed(0)
np.random.seed(0)

from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.optimizers import SGD, Adam
from keras.callbacks import EarlyStopping
from sklearn.metrics import precision_recall_fscore_support, roc_auc_score
from keras.utils import np_utils

frames = 41
bands = 60
feature_size = bands * frames #60x41
num_labels = 10
num_channels = 2

Using TensorFlow backend.


This method defines some evaluation metrics that will be used to evaluate the performance of a trained model.

In [19]:
def evaluate(model):
    y_prob = model.predict_proba(test_x, verbose=0)
    y_pred = y_prob.argmax(axis=-1)
    y_true = np.argmax(test_y, 1)

    roc = roc_auc_score(test_y, y_prob)
    print ("ROC:",  round(roc,3))

    # evaluate the model
    score, accuracy = model.evaluate(test_x, test_y, batch_size=32)
    print("\nAccuracy = {:.2f}".format(accuracy))

    # the F-score gives a similiar value to the accuracy score, but useful for cross-checking
    p,r,f,s = precision_recall_fscore_support(y_true, y_pred, average='micro')
    print ("F-Score:", round(f,2))
    
    return roc, accuracy

The following section defines the successive layers of our convolutional neural network (CNN). 

CNNs differ from the FFN described in notebook 2 by having dedicated convolutional layers between the input layer and the hidden layers. It begins with two Convolution2D layers, the first with 96 output filters, the second with 64 - the idea being to apply convolutions that will progressively squeeze the spatial dimensions while increasing the depth. In effect, space is transformed into depth, spatial information is removed until only semantic complexity remains. 

Once a deep and narrow representation has been created, the output is fed into a regular deep neural network of densely connected layers, and used to train a classifier just as we did with the Feed-Forward Network.

As well as familiar techniques like Dropout and Activation Functions, you'll notice something new in this model - a 
process called Max Pooling. This works by looking at a small neighbourhood around every point in the future map, and computing the maximum of all the responses around it. Combining many weaker signals into one stronger signal helps reduce the risk of over-fitting.


In [24]:
def build_model():
    
    model = Sequential()
    # input: 60x41 data frames with 2 channels => (60,41,2) tensors

    # filters of size 1x1 
    f_size = 1

    # first layer has 48 convolution filters 
    model.add(Convolution2D(48, f_size, strides=f_size, kernel_initializer='normal', padding='same', input_shape=(bands, frames, num_channels)))
    model.add(Convolution2D(48, f_size, strides=f_size, kernel_initializer='normal', padding='same'))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.5))

    # next layer has 96 convolution filters
    model.add(Convolution2D(96, f_size, strides=f_size, kernel_initializer='normal', padding='same'))
    model.add(Convolution2D(96, f_size, strides=f_size, kernel_initializer='normal', padding='same'))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.5))

    # flatten output into a single dimension 
    # Keras will do shape inference automatically
    model.add(Flatten())

    # then a fully connected NN layer
    model.add(Dense(256))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))

    # finally, an output layer with one node per class
    model.add(Dense(num_labels))
    model.add(Activation('softmax'))

    # use the Adam optimiser
    adam = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0)
    model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer=adam)
    
    return model

### Training on a minimised data set

The data set is a large quantity of data, which is quite computationally intensive to process. But if you've checked out the git repository for this project, I've included a selection from the original data set, which should allow you to run the CNN and see how it works. Obviously as this training set size is much smaller and contains much less diversity than the full set, lower accuracies will be achievable when models are trained on it, but it will allow you to run the code and explore how it works. 

In [26]:
data_dir = "data/us8k-np-cnn-mini"

# load fold1 for testing
train_x, train_y = load_folds([1])

# load fold2 for validation
valid_x, valid_y = load_folds([2])
    
# load fold3 for testing
test_x, test_y = load_folds([3])

print("Building model...")
model = build_model()

# a stopping function to stop training before we excessively overfit to the training set
earlystop = EarlyStopping(monitor='val_loss', patience=0, verbose=1, mode='auto')

# now fit the model to the training data, evaluating loss against the validation data
print("Training model...")
model.fit(train_x, train_y, validation_data=(valid_x, valid_y), callbacks=[earlystop], batch_size=20, epochs=3)

# now evaluate the trained model against the unseen test data
print("Evaluating model...")
roc, acc = evaluate(model)


fold1 features:  (2400, 60, 41, 2)
fold2 features:  (2076, 60, 41, 2)
fold3 features:  (2371, 60, 41, 2)
Building model...
Training model...
Train on 2400 samples, validate on 2076 samples
Epoch 1/3
2400/2400 [==============================] - 47s - loss: 1.7494 - acc: 0.3550 - val_loss: 1.8051 - val_acc: 0.3921
Epoch 2/3
2400/2400 [==============================] - 48s - loss: 1.3732 - acc: 0.5050 - val_loss: 1.8185 - val_acc: 0.4027
Epoch 00001: early stopping
Evaluating model...
ROC: 0.737
2368/2371 [============================>.] - ETA: 0s
Accuracy = 0.39
F-Score: 0.39


### Training on the full data set (intensive) 

To run this code you'll need to have downloaded and run the feature extraction process on the entire UrbanSound8k data set (which requires some quite intensive processing).

The training process below is an implementation of multi-fold cross-validation, it will create a fresh model for each trial, load the next fold of input data, fit it to the model and evaluate the accuracy achieved. When all the input data has been used, we can calculate the average accuracy for all the folds.

In [28]:
data_dir = "data/us8k-np-cnn"
all_folds = False
av_acc = 0.
av_roc = 0.
num_folds = 0

# as we use two folds for training, there are 9 possible trails rather than 10
max_trials = 5

# earlystopping ends training when the validation loss stops improving
earlystop = EarlyStopping(monitor='val_loss', patience=0, verbose=0, mode='auto')

if all_folds:
    load_all_folds()
    model.fit(train_x, train_y, validation_data=(valid_x, valid_y), callbacks=[earlystop], batch_size=32, epochs=1)
else:
    # use folds incrementally
    for f in range(1,max_trials+1):
        num_folds += 1
        v = f + 2
        if v > 10: v = 1
        t = v + 1
        if t > 10: t = 1
        
        print ("\n*** Train on", f, "&",(f+1), "Validate on", v, "Test on", t, "***")
    
        # load two folds for training data
        train_x, train_y = load_folds([f,f+1])
    
        # load one fold for validation
        valid_x, valid_y = load_folds([v])
    
        # load one fold for testing
        test_x, test_y = load_folds([t])
        
        print("Building model...")
        model = build_model()

        # now fit the model to the training data, evaluating loss against the validation data
        print("Training model...")
        model.fit(train_x, train_y, validation_data=(valid_x, valid_y), callbacks=[earlystop], batch_size=64, epochs=1)

        # now evaluate the trained model against the unseen test data
        print("Evaluating model...")
        roc, acc = evaluate(model)
        av_roc += roc
        av_acc += acc
    
print ('\nAverage R.O.C:', round(av_roc/max_trials, 3))
print ('Average Accuracy:', round(av_acc/max_trials, 3))

# using all folds: best ROC = 0.889, f-score = 0.591
# using 2 folds: average ROC = 0.792, average f-score = 0.335


*** Train on 1 & 2 Validate on 3 Test on 4 ***
fold1 features:  (5446, 60, 41, 2)
fold2 features:  (5388, 60, 41, 2)
fold3 features:  (5852, 60, 41, 2)
fold4 features:  (6048, 60, 41, 2)
Building model...
Training model...


/Users/jaron/anaconda/envs/tflearn/lib/python3.5/site-packages/keras/models.py:844: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Train on 10834 samples, validate on 5852 samples
Epoch 1/1
10834/10834 [==============================] - 163s - loss: 1.7122 - acc: 0.3662 - val_loss: 1.9999 - val_acc: 0.3337
Evaluating model...
ROC: 0.726
6048/6048 [==============================] - 18s    

Accuracy = 0.33
F-Score: 0.33

*** Train on 2 & 3 Validate on 4 Test on 5 ***
fold2 features:  (5388, 60, 41, 2)
fold3 features:  (5852, 60, 41, 2)
fold4 features:  (6048, 60, 41, 2)
fold5 features:  (5689, 60, 41, 2)
Building model...
Training model...
Train on 11240 samples, validate on 6048 samples
Epoch 1/1
11240/11240 [==============================] - 181s - loss: 1.7266 - acc: 0.3722 - val_loss: 1.8495 - val_acc: 0.3833
Evaluating model...
ROC: 0.722
5689/5689 [==============================] - 22s    

Accuracy = 0.31
F-Score: 0.31

*** Train on 3 & 4 Validate on 5 Test on 6 ***
fold3 features:  (5852, 60, 41, 2)
fold4 features:  (6048, 60, 41, 2)
fold5 features:  (5689, 60, 41, 2)
fold6 features:  (5080, 60, 41, 2)
Buil

### Generating Predictions 

Now, test the prediction performance.

In [29]:
sound_file_paths = ["aircon.wav", "carhorn.wav", "play.wav", "dogbark.wav", "drill.wav",
                    "engine.wav","gunshots.wav","jackhammer.wav","siren.wav","music.wav"]
sound_names = ["air conditioner","car horn","children playing","dog bark","drilling","engine idling",
               "gun shot","jackhammer","siren","street music"]
parent_dir = 'samples/us8k/'



# create predictions for each of the sound classes
for s in range(len(sound_names)):

    print ("\n----- ", sound_names[s], "-----")
    # load audio file and extract features
    predict_file = parent_dir + sound_file_paths[s]
    predict_x = extract_feature_array(predict_file)
    
    # generate prediction, passing in just a single row of features
    predictions = model.predict(predict_x)
    
    if len(predictions) == 0: 
        print ("No prediction")
        continue
    
    #for i in range(len(predictions[0])):
    #    print sound_names[i], "=", round(predictions[0,i] * 100, 1)
    
    # get the indices of the top 2 predictions, invert into descending order
    ind = np.argpartition(predictions[0], -2)[-2:]
    ind[np.argsort(predictions[0][ind])]
    ind = ind[::-1]
    
    print ("Top guess: ", sound_names[ind[0]], " (",round(predictions[0,ind[0]],3),")")
    print ("2nd guess: ", sound_names[ind[1]], " (",round(predictions[0,ind[1]],3),")")

    


-----  air conditioner -----
Top guess:  air conditioner  ( 0.304 )
2nd guess:  children playing  ( 0.202 )

-----  car horn -----
Top guess:  car horn  ( 0.403 )
2nd guess:  drilling  ( 0.155 )

-----  children playing -----
Top guess:  jackhammer  ( 0.33 )
2nd guess:  children playing  ( 0.19 )

-----  dog bark -----
Top guess:  children playing  ( 0.349 )
2nd guess:  street music  ( 0.228 )

-----  drilling -----
Top guess:  jackhammer  ( 0.768 )
2nd guess:  drilling  ( 0.174 )

-----  engine idling -----
Top guess:  street music  ( 0.505 )
2nd guess:  engine idling  ( 0.214 )

-----  gun shot -----
Top guess:  drilling  ( 0.166 )
2nd guess:  air conditioner  ( 0.156 )

-----  jackhammer -----
Top guess:  jackhammer  ( 0.398 )
2nd guess:  drilling  ( 0.186 )

-----  siren -----
Top guess:  siren  ( 0.727 )
2nd guess:  engine idling  ( 0.192 )

-----  street music -----
Top guess:  drilling  ( 0.253 )
2nd guess:  engine idling  ( 0.241 )


The accuracy score gives a measure of the model's predictive power over the entire test set, but it can also be useful to look at how the model behaves at the smallest scale, when asked to predict the class of individual files. 

If you run the code above, you'll see the predicted classes for each of the sample files, 6 of the 10 predictions are correct, but the other 4 are incorrect. If you listen to the sample files, you might be able to identify most of them, but some are ambiguous, such as air conditioning that sounds like distant traffic. 

The best accuracy I have achieved with the code in this notebook was ~59% - and that was using the full UrbanSound8K data set. So clearly audio classification, like image classification, is not a trivial challenge. The next notebook (#4) will describe the implementation of a more sophisticated CNN, with the aim of achieving even better classification performance.